# 1. Import Libraries

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

from tensorflow.keras import Sequential, Input
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical

2025-04-23 23:30:34.910816: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-23 23:30:34.913794: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-23 23:30:34.921805: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745425834.939611   30330 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745425834.943940   30330 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745425834.955749   30330 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

# 2. Dataset

In [2]:
data = pd.read_csv('datasets/Dataset2/train.csv')
test = pd.read_csv('datasets/Dataset2/test.csv')

data.info()
data.head()
data.describe().T

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   battery_power  2000 non-null   int64  
 1   blue           2000 non-null   int64  
 2   clock_speed    2000 non-null   float64
 3   dual_sim       2000 non-null   int64  
 4   fc             2000 non-null   int64  
 5   four_g         2000 non-null   int64  
 6   int_memory     2000 non-null   int64  
 7   m_dep          2000 non-null   float64
 8   mobile_wt      2000 non-null   int64  
 9   n_cores        2000 non-null   int64  
 10  pc             2000 non-null   int64  
 11  px_height      2000 non-null   int64  
 12  px_width       2000 non-null   int64  
 13  ram            2000 non-null   int64  
 14  sc_h           2000 non-null   int64  
 15  sc_w           2000 non-null   int64  
 16  talk_time      2000 non-null   int64  
 17  three_g        2000 non-null   int64  
 18  touch_sc

,count,mean,std,min,25%,50%,75%,max
battery_power,2000.0,1238.51850,439.418206,501.0,851.75,1226.0,1615.25,1998.0
blue,2000.0,0.49500,0.500100,0.0,0.00,0.0,1.00,1.0
clock_speed,2000.0,1.52225,0.816004,0.5,0.70,1.5,2.20,3.0
dual_sim,2000.0,0.50950,0.500035,0.0,0.00,1.0,1.00,1.0
fc,2000.0,4.30950,4.341444,0.0,1.00,3.0,7.00,19.0
four_g,2000.0,0.52150,0.499662,0.0,0.00,1.0,1.00,1.0
int_memory,2000.0,32.04650,18.145715,2.0,16.00,32.0,48.00,64.0
m_dep,2000.0,0.50175,0.288416,0.1,0.20,0.5,0.80,1.0
mobile_wt,2000.0,140.24900,35.399655,80.0,109.00,141.0,170.00,200.0
n_cores,2000.0,4.52050,2.287837,1.0,3.00,4.0,7.00,8.0


# 3. Data Preprocessing

In [3]:
# Drop id
for df in [data, test]:
    if 'id' in df.columns:
        df.drop('id', axis=1, inplace=True)

# Pisah fitur dan target
y = data['price_range']
X = data.drop('price_range', axis=1)

# Split 70:30
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

# Scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 4. Method

## 4.1. SVM

In [4]:
svm = SVC(kernel='rbf', decision_function_shape='ovr')
svm.fit(X_train, y_train)

y_pred = svm.predict(X_test)

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.93      0.95       150
           1       0.85      0.83      0.84       150
           2       0.77      0.83      0.79       150
           3       0.90      0.88      0.89       150

    accuracy                           0.87       600
   macro avg       0.87      0.87      0.87       600
weighted avg       0.87      0.87      0.87       600

[[140  10   0   0]
 [  5 125  20   0]
 [  0  12 124  14]
 [  0   0  18 132]]


## 4.2. MLP

In [5]:
mlp = MLPClassifier(hidden_layer_sizes=(100,50), max_iter=200, random_state=42)
mlp.fit(X_train, y_train)

y_pred = mlp.predict(X_test)

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.91      0.93       150
           1       0.87      0.89      0.88       150
           2       0.89      0.88      0.89       150
           3       0.92      0.96      0.94       150

    accuracy                           0.91       600
   macro avg       0.91      0.91      0.91       600
weighted avg       0.91      0.91      0.91       600

[[136  14   0   0]
 [  7 133  10   0]
 [  0   6 132  12]
 [  0   0   6 144]]


## 4.3. CNN

In [6]:
# Reshape for CNN-1D
timesteps = X_train.shape[1]

X_train_cnn = X_train.reshape(-1, timesteps, 1)
X_test_cnn = X_test.reshape(-1, timesteps, 1)

y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

# Define CNN model
model_cnn = Sequential([
    Input(shape=(timesteps, 1)),
    Conv1D(32, 3, activation='relu'),
    Dropout(0.3),
    Conv1D(64, 3, activation='relu'),
    Flatten(),
    Dense(100, activation='relu'),
    Dense(y_train_cat.shape[1], activation='softmax')
])

# Compile and train
model_cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_cnn.fit(X_train_cnn, y_train_cat, epochs=25, batch_size=32, validation_split=0.2)

# Evaluate
y_cnn_pred = np.argmax(model_cnn.predict(X_test_cnn), axis=1)
print(confusion_matrix(y_test, y_cnn_pred))

Epoch 1/25


E0000 00:00:1745425837.519695   30330 cuda_executor.cc:1228] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1745425837.524275   30330 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.4059 - loss: 1.2881 - val_accuracy: 0.7214 - val_loss: 0.8234
Epoch 2/25
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7242 - loss: 0.7221 - val_accuracy: 0.7714 - val_loss: 0.5328
Epoch 3/25
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7632 - loss: 0.5370 - val_accuracy: 0.8393 - val_loss: 0.3817
Epoch 4/25
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8460 - loss: 0.3868 - val_accuracy: 0.8821 - val_loss: 0.3205
Epoch 5/25
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8640 - loss: 0.3478 - val_accuracy: 0.8500 - val_loss: 0.3375
Epoch 6/25
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8463 - loss: 0.3211 - val_accuracy: 0.8643 - val_loss: 0.2962
Epoch 7/25
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8627 - loss: 0.3382 - val_accuracy: 0.8714 - val_loss: 0.2968
Epoch 8/25
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8992 - loss: 0.2757 - val_accuracy: 0.8964 - val_loss: 0.2718
Epo

In [7]:
# Initialize SVM with probability=True
svm = SVC(kernel='rbf', decision_function_shape='ovr', probability=True)
svm.fit(X_train, y_train)

# Get probabilities for ROC-AUC
probs = svm.predict_proba(X_test)
print("SVM Multi-class ROC-AUC (OVR):", roc_auc_score(y_test, probs, multi_class='ovr'))

SVM Multi-class ROC-AUC (OVR): 0.9799148148148149
